# Creating a netCDF file with eNd for atmospheric flux
netCDF file on a FAMOUS grid, containing eNd values across the ocean for the atmospheric flux
just setting whole ocean to a background eNd value of -8.0 

creating a .nc file which i will translate into an ancil file 
from which I will create a mod to convert this into a dust flux of 144Nd and 143Nd

based on examples from:
https://iescoders.com/2017/10/03/writing-netcdf4-data-in-python/
http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/11_create_netcdf_python.pdf

In [1]:
# load packages

%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend
import matplotlib.gridspec as gridspec # subplots
from matplotlib import colors as mcolors

import iris
import iris.plot as iplt
import iris.quickplot as qplt
#plt.interactive(True)

import numpy as np
import pandas as pd

In [2]:
from netCDF4 import Dataset

#this is an nc file with salinity from which to base my template on
f=Dataset('/nfs/see-fs-01_users/ee14s2r/scripts/python/surf_flux/qrparm.waterfix.famous.nc','r')

#latitude in and longitude in
latin = f.variables['latitude'][:]
latsize=len(latin)
lonin = f.variables['longitude'][:]
lonsize=len(lonin)

#if xconv qrparm.waterfix.famous.nc then it will have the field number
waterflux=f.variables['field672'][:]
#This np.where is changing values of salnity into background eNd values and then changing the land mask to 0
waterflux2=np.where(waterflux>1.0E20,0.0,-8.0)

print(waterflux2)
print(np.shape(waterflux2))

#open a new netCDF file to create my file for inputing 144Nd surface flux
dataset= Dataset('/nfs/see-fs-01_users/ee14s2r/working/surf_files/eNd_dust_mask/dust_flux_background_eNd.nc', 'w',
                 format='NETCDF4_CLASSIC')

print(dataset.file_format)

[[[[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [-8. -8. -8. ..., -8. -8. -8.]
   [-8. -8. -8. ..., -8. -8. -8.]
   [ 0.  0.  0. ...,  0.  0.  0.]]]]
(1, 1, 73, 98)
NETCDF4_CLASSIC


In [3]:
#Define dimensions used for my variables
#As i am imposing a surface flux then i just want to add it at depth 1
depth = dataset.createDimension("depth", 1)
lat = dataset.createDimension("lat", latsize)
lon = dataset.createDimension("lon", lonsize)
time = dataset.createDimension("time", 1)

In [4]:
#Checking dimensions
print (len(lon))
print (time.isunlimited())

98
False


In [5]:
#Creating a variable
#Define variables with the same names as dimensions
#Create coordinate variables for 4-dimensions

depth = dataset.createVariable("depth", np.int32, ("depth",))
depth.units = 'm'
depth.positive= 'down'
lat = dataset.createVariable("lat", np.float32, ("lat",))
lat.units = 'degrees_north'
lat.point_spacing = 'even'
lat.long_name = 'latitude'
lon = dataset.createVariable("lon", np.float32, ("lon",))
lon.units = 'degrees_east'
lon.point_spacing = 'even'
lon.long_name = 'longitude'
time = dataset.createVariable("time", np.float64, ("time",))
time.units = ('days since 0000-01-01 00:00.00') 

#Create the 4d-variable for eNd
eNd = dataset.createVariable("eNd", np.float32, ("time","depth","lat","lon"))
eNd.units = 'ratio'
print('eNd144')

eNd144


In [6]:
#Accessing and checking variables
for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape

depth int32 (u'depth',) (1,)
lat float32 (u'lat',) (73,)
lon float32 (u'lon',) (98,)
time float64 (u'time',) (1,)
eNd float32 (u'time', u'depth', u'lat', u'lon') (1, 1, 73, 98)


In [7]:
#Writing data, slicing it into lon and lat

lat[:] = latin
lon[:] = lonin
eNd[:,:,:,:]=waterflux2
print(np.shape(eNd))
print(np.shape(waterflux2))
print(eNd)


(1, 1, 73, 98)
(1, 1, 73, 98)
<type 'netCDF4._netCDF4.Variable'>
float32 eNd(time, depth, lat, lon)
    units: ratio
unlimited dimensions: 
current shape = (1, 1, 73, 98)
filling on, default _FillValue of 9.96920996839e+36 used



In [8]:
#Define a box in the North Atlantic and give it a value of eNd=-15.0
#Defining my North Atlantic box as : 
#latitude (degrees north): [-90 - -50]
#longitude (degrees east): [127.5 - 251.25]

#N_Atlantic=

In [9]:
dataset.close()